# Azure AI Foundry

<center><img src="../../../images/Azure-AI-Foundry_1600x900.jpg" alt="Azure AI Foundry" width="600">

## Laboratório 3

### Prompt Engineering

Prompt Engineering é a arte e ciência de criar instruções eficazes para modelos de linguagem de grande escala (LLMs). É uma habilidade essencial para obter resultados precisos, relevantes e úteis ao interagir com IA generativa.

**O que é Prompt Engineering?**

Prompt Engineering envolve o design cuidadoso de instruções (prompts) que orientam o comportamento de modelos de IA para produzir saídas desejadas. Isso inclui a escolha de palavras, estruturação de informações, fornecimento de contexto e definição de formatos de saída.

**Por que é Importante?**

- **Precisão**: Prompts bem elaborados produzem resultados mais precisos
- **Consistência**: Técnicas estruturadas garantem resultados previsíveis
- **Eficiência**: Reduz a necessidade de múltiplas tentativas
- **Controle**: Permite maior controle sobre o estilo e formato da saída

**Componentes de um Bom Prompt:**

1. **Contexto**: Informações de fundo relevantes
2. **Instrução**: O que você quer que o modelo faça
3. **Exemplos**: Demonstrações do formato desejado (quando aplicável)
4. **Limitações**: Restrições ou diretrizes específicas
5. **Formato de Saída**: Como a resposta deve ser estruturada

Nas seções seguintes, exploraremos várias técnicas avançadas de Prompt Engineering:

In [ ]:
# Configuração do Azure OpenAI
import json
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Carrega as variáveis de ambiente
load_dotenv(dotenv_path="../../../.env")

# Configurações
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")

# Inicializa o cliente
client = AzureOpenAI(
    azure_endpoint=azure_endpoint, 
    api_key=api_key,  
    api_version=api_version
)

print("Cliente Azure OpenAI configurado com sucesso!")

### Zero-Shot Prompting

Zero-Shot Prompting é uma técnica onde você fornece uma tarefa ao modelo de linguagem sem exemplos prévios. O modelo deve entender e executar a tarefa baseado apenas em sua compreensão geral e conhecimento pré-treinado.

**Características:**
- Não requer exemplos de entrada/saída
- Depende do conhecimento pré-treinado do modelo
- Simples de implementar
- Pode não ser eficaz para tarefas complexas ou específicas

**Exemplo:**
```
Prompt: "Traduza a seguinte frase para o inglês: 'Olá, como você está?'"
Resposta: "Hello, how are you?"
```

**Quando usar:**
- Tarefas simples e bem definidas
- Quando você não tem exemplos disponíveis
- Para teste inicial de capacidades do modelo

In [ ]:
# Exemplo prático de Zero-Shot Prompting
print("=== TESTE: Zero-Shot Prompting ===")

# Exemplo 1: Tradução
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": "Traduza a seguinte frase para o inglês: 'Olá, como você está?'"}
    ],
    temperature=0.3
)

print("Tradução:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Exemplo 2: Classificação de sentimento
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": "Classifique o sentimento desta frase como positivo, negativo ou neutro: 'Este produto é incrível, recomendo muito!'"}
    ],
    temperature=0.1
)

print("Classificação de sentimento:")
print(response.choices[0].message.content)

### Few-Shot Prompting

Few-Shot Prompting é uma técnica onde você fornece alguns exemplos de entrada e saída esperada antes de apresentar a tarefa real. Isso ajuda o modelo a entender melhor o padrão e formato desejado da resposta.

**Características:**
- Inclui 2-5 exemplos demonstrativos
- Melhora a precisão em comparação ao zero-shot
- Ajuda o modelo a entender o formato de saída
- Eficaz para tarefas que seguem padrões específicos

**Exemplo:**
```
Prompt: 
"Classifique o sentimento das seguintes frases:
Frase: 'Eu amo este produto!' → Sentimento: Positivo
Frase: 'Este serviço é terrível.' → Sentimento: Negativo
Frase: 'O produto está ok.' → Sentimento: Neutro
Frase: 'Estou muito feliz com a compra!' → Sentimento: ?"
```

**Quando usar:**
- Tarefas que requerem formato específico
- Quando zero-shot não fornece resultados adequados
- Para classificação ou tarefas estruturadas

In [ ]:
# Exemplo prático de Few-Shot Prompting
print("=== TESTE: Few-Shot Prompting ===")

# Exemplo: Classificação de sentimentos com exemplos
few_shot_prompt = """Classifique o sentimento das seguintes frases:

Frase: 'Eu amo este produto!' → Sentimento: Positivo
Frase: 'Este serviço é terrível.' → Sentimento: Negativo  
Frase: 'O produto está ok.' → Sentimento: Neutro
Frase: 'Não consegui usar o aplicativo, muito confuso.' → Sentimento: Negativo
Frase: 'Funcionou perfeitamente, exatamente como esperado!' → Sentimento: Positivo

Frase: 'O atendimento foi razoável, nada demais.' → Sentimento: ?"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": few_shot_prompt}
    ],
    temperature=0.1
)

print("Classificação com Few-Shot:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Exemplo 2: Formatação de dados com exemplos
format_prompt = """Converta os dados para o formato especificado:

Nome: João Silva, Idade: 30, Cidade: São Paulo → {"nome": "João Silva", "idade": 30, "cidade": "São Paulo"}
Nome: Maria Santos, Idade: 25, Cidade: Rio de Janeiro → {"nome": "Maria Santos", "idade": 25, "cidade": "Rio de Janeiro"}
Nome: Pedro Costa, Idade: 45, Cidade: Belo Horizonte → {"nome": "Pedro Costa", "idade": 45, "cidade": "Belo Horizonte"}

Nome: Ana Oliveira, Idade: 28, Cidade: Brasília → ?"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": format_prompt}
    ],
    temperature=0.1
)

print("Formatação com Few-Shot:")
print(response.choices[0].message.content)

### Chain-of-Thought Prompting

Chain-of-Thought (CoT) Prompting é uma técnica que encoraja o modelo a mostrar seu raciocínio passo a passo antes de chegar à resposta final. Isso melhora significativamente a performance em tarefas de raciocínio complexo.

**Características:**
- Quebra problemas complexos em etapas menores
- Mostra o processo de raciocínio
- Melhora a precisão em problemas matemáticos e lógicos
- Permite identificar onde o raciocínio pode ter falhado

**Exemplo:**
```
Prompt: "Resolva passo a passo: Se um trem viaja a 60 km/h e precisa percorrer 180 km, quanto tempo levará?

Vamos pensar passo a passo:
1. Velocidade = 60 km/h
2. Distância = 180 km
3. Tempo = Distância ÷ Velocidade
4. Tempo = 180 ÷ 60 = 3 horas

Resposta: 3 horas"
```

**Quando usar:**
- Problemas matemáticos ou lógicos
- Tarefas que requerem raciocínio multi-etapas
- Quando você precisa verificar o processo de pensamento

In [ ]:
# Exemplo prático de Chain-of-Thought Prompting
print("=== TESTE: Chain-of-Thought Prompting ===")

# Exemplo 1: Problema matemático
cot_prompt = """Resolva passo a passo: 

Em uma loja, há 24 camisetas. 1/3 delas são azuis, 1/4 são vermelhas e o restante são brancas. 
Se o preço de cada camiseta azul é R$ 30, cada vermelha R$ 25 e cada branca R$ 20, qual é o valor total do estoque?

Vamos pensar passo a passo:"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": cot_prompt}
    ],
    temperature=0.1
)

print("Solução com Chain-of-Thought:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Exemplo 2: Problema lógico
logic_prompt = """Resolva este problema de lógica passo a passo:

Ana, Bruno e Carlos estão em uma fila. 
- Ana não está na frente
- Bruno não está no meio
- Carlos não está atrás

Qual é a ordem da fila?

Vamos analisar cada pista:"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": logic_prompt}
    ],
    temperature=0.1
)

print("Solução lógica com Chain-of-Thought:")
print(response.choices[0].message.content)

### Meta Prompting

Meta Prompting é uma técnica avançada onde o modelo é instruído a gerar ou melhorar prompts. Essencialmente, você usa o modelo para criar prompts melhores para ele mesmo ou para outras tarefas.

**Características:**
- O modelo ajuda a criar prompts mais eficazes
- Pode melhorar iterativamente a qualidade dos prompts
- Útil para otimização automática de prompts
- Requer conhecimento sobre técnicas de prompting

**Exemplo:**
```
Prompt: "Crie um prompt eficaz para fazer um modelo de IA explicar conceitos científicos complexos para crianças de 10 anos. O prompt deve incluir:
- Linguagem simples
- Analogias adequadas para a idade
- Estrutura clara
- Elementos interativos"

Resposta: "Explique [conceito científico] para uma criança de 10 anos. Use palavras simples, compare com coisas que ela conhece do dia a dia, organize em 3 partes principais e faça perguntas para manter o interesse."
```

**Quando usar:**
- Otimização de prompts existentes
- Criação de prompts para tarefas específicas
- Quando você precisa de múltiplas variações de um prompt

In [ ]:
# Exemplo prático de Meta Prompting
print("=== TESTE: Meta Prompting ===")

# Exemplo 1: Criação de prompt para ensino
meta_prompt = """Crie um prompt eficaz para fazer um modelo de IA explicar conceitos de programação para iniciantes. O prompt deve incluir:
- Linguagem simples e acessível
- Analogias com o mundo real
- Estrutura clara e didática  
- Exemplos práticos
- Verificação de compreensão

Forneça apenas o prompt otimizado, sem explicações adicionais."""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": meta_prompt}
    ],
    temperature=0.7
)

generated_prompt = response.choices[0].message.content
print("Prompt gerado pelo Meta Prompting:")
print(generated_prompt)
print("\n" + "="*50 + "\n")

# Exemplo 2: Testando o prompt gerado
test_content = generated_prompt + "\n\nConceito: Variáveis em programação"

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": test_content}
    ],
    temperature=0.5
)

print("Teste do prompt gerado:")
print(response.choices[0].message.content)

### Prompt Chaining

Prompt Chaining é uma técnica onde você quebra uma tarefa complexa em múltiplos prompts sequenciais, onde a saída de um prompt serve como entrada para o próximo.

**Características:**
- Divide tarefas complexas em etapas menores
- A saída de um prompt alimenta o próximo
- Permite maior controle sobre cada etapa
- Reduz a chance de erros em tarefas complexas

**Exemplo:**
```
Prompt 1: "Analise este texto e identifique os pontos principais: [texto]"
Saída 1: "Pontos principais: A, B, C"

Prompt 2: "Com base nos pontos principais: A, B, C, crie um resumo executivo de 100 palavras"
Saída 2: [Resumo executivo]

Prompt 3: "Transforme este resumo em uma apresentação de 3 slides: [resumo]"
```

**Quando usar:**
- Tarefas que envolvem múltiplas etapas de processamento
- Quando você precisa de controle granular sobre cada fase
- Para transformação de dados em múltiplos formatos
- Análise complexa que requer validação em cada etapa

In [ ]:
# Exemplo prático de Prompt Chaining
print("=== TESTE: Prompt Chaining ===")

# Texto de exemplo para análise
texto_exemplo = """
A inteligência artificial está transformando rapidamente diversos setores da economia. 
Na área da saúde, algoritmos de machine learning estão sendo usados para diagnósticos 
mais precisos e descoberta de novos medicamentos. No setor financeiro, a IA ajuda na 
detecção de fraudes e análise de riscos. Na educação, sistemas inteligentes personalizam 
o aprendizado para cada estudante. Entretanto, também surgem desafios éticos e questões 
sobre o futuro do trabalho.
"""

# Prompt 1: Identificar pontos principais
print("1. Identificando pontos principais...")
response1 = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": f"Analise este texto e identifique os 3 pontos principais em formato de lista:\n\n{texto_exemplo}"}
    ],
    temperature=0.3
)

pontos_principais = response1.choices[0].message.content
print("Pontos principais identificados:")
print(pontos_principais)
print("\n" + "-"*30 + "\n")

# Prompt 2: Criar resumo executivo
print("2. Criando resumo executivo...")
response2 = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": f"Com base nos pontos principais identificados:\n{pontos_principais}\n\nCrie um resumo executivo de exatamente 50 palavras."}
    ],
    temperature=0.3
)

resumo_executivo = response2.choices[0].message.content
print("Resumo executivo:")
print(resumo_executivo)
print("\n" + "-"*30 + "\n")

# Prompt 3: Criar apresentação
print("3. Criando estrutura de apresentação...")
response3 = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": f"Transforme este resumo em uma estrutura de apresentação de 3 slides com títulos e tópicos:\n\n{resumo_executivo}"}
    ],
    temperature=0.5
)

print("Estrutura da apresentação:")
print(response3.choices[0].message.content)

### Tree of Thoughts (ToT)

Tree of Thoughts (ToT) é uma técnica avançada que permite ao modelo explorar múltiplos caminhos de raciocínio simultaneamente, como uma árvore de decisão, avaliando e escolhendo os melhores caminhos.

**Características:**
- Explora múltiplas abordagens simultaneamente
- Permite backtracking quando um caminho não funciona
- Avalia a qualidade de cada "pensamento" ou etapa
- Mais robusto que Chain-of-Thought linear

**Exemplo:**
```
Problema: "Como resolver o conflito entre equipes?"

Pensamento 1: Mediação direta
├── Avaliação: Pode ser confrontativo
├── Próximo passo: Reunião formal
└── Resultado esperado: Resolução rápida

Pensamento 2: Facilitação indireta
├── Avaliação: Menos estressante
├── Próximo passo: Conversas individuais
└── Resultado esperado: Compreensão gradual

Pensamento 3: Reestruturação de processos
├── Avaliação: Solução sistemática
├── Próximo passo: Análise de workflows
└── Resultado esperado: Prevenção futura

Melhor caminho: Combinação de 2 e 3
```

**Quando usar:**
- Problemas complexos com múltiplas soluções possíveis
- Quando você precisa explorar alternativas
- Planejamento estratégico e tomada de decisão

In [ ]:
# Exemplo prático de Tree of Thoughts (ToT)
print("=== TESTE: Tree of Thoughts (ToT) ===")

# Problema complexo para análise
tot_prompt = """Problema: Uma empresa de tecnologia está enfrentando alta rotatividade de funcionários (30% ao ano). 
Como CEO, você precisa resolver isso rapidamente.

Explore 3 abordagens diferentes simultaneamente:

ABORDAGEM 1 - Melhoria Salarial:
- Avalie: Prós, contras e viabilidade
- Próximos passos específicos
- Resultado esperado

ABORDAGEM 2 - Melhoria do Ambiente de Trabalho:
- Avalie: Prós, contras e viabilidade  
- Próximos passos específicos
- Resultado esperado

ABORDAGEM 3 - Programa de Desenvolvimento:
- Avalie: Prós, contras e viabilidade
- Próximos passos específicos
- Resultado esperado

ANÁLISE FINAL:
- Compare as 3 abordagens
- Recomende a melhor estratégia ou combinação
- Justifique sua escolha"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": tot_prompt}
    ],
    temperature=0.7,
    max_tokens=1000
)

print("Análise Tree of Thoughts:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Exemplo 2: Problema de planejamento
planning_prompt = """Você precisa planejar o lançamento de um novo produto mobile app em 6 meses.
Explore simultaneamente 3 estratégias de go-to-market:

ESTRATÉGIA A - Lançamento Gradual:
- Análise de viabilidade: 
- Recursos necessários:
- Riscos e mitigações:

ESTRATÉGIA B - Lançamento Massivo:
- Análise de viabilidade:
- Recursos necessários: 
- Riscos e mitigações:

ESTRATÉGIA C - Lançamento por Nicho:
- Análise de viabilidade:
- Recursos necessários:
- Riscos e mitigações:

DECISÃO: Escolha a melhor estratégia baseado na análise."""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": planning_prompt}
    ],
    temperature=0.6,
    max_tokens=800
)

print("Planejamento com Tree of Thoughts:")
print(response.choices[0].message.content)

### Retrieval Augmented Generation (RAG)

Retrieval Augmented Generation (RAG) é uma técnica que combina a geração de texto do modelo com a recuperação de informações de uma base de conhecimento externa, permitindo respostas mais precisas e atualizadas.

**Características:**
- Combina geração com recuperação de informações
- Acessa conhecimento externo e atualizado
- Reduz alucinações do modelo
- Permite citar fontes específicas

**Componentes do RAG:**
1. **Base de Conhecimento**: Documentos, artigos, bases de dados
2. **Sistema de Recuperação**: Busca informações relevantes
3. **Gerador**: Modelo de linguagem que cria a resposta
4. **Integração**: Combina informações recuperadas com geração

**Exemplo de Processo:**
```
Pergunta: "Quais são as novidades do Azure AI em 2024?"

1. Recuperação: Busca documentos recentes sobre Azure AI
2. Contexto: "Azure AI Foundry foi lançado em 2024..."
3. Prompt: "Com base nas informações: [contexto], responda: [pergunta]"
4. Resposta: Gerada com base no contexto recuperado
```

**Quando usar:**
- Quando você precisa de informações atualizadas
- Para reduzir alucinações
- Em sistemas de Q&A corporativos
- Quando o modelo precisa citar fontes específicas

In [ ]:
# Exemplo prático de RAG (Simulado)
print("=== TESTE: Retrieval Augmented Generation (RAG) ===")

# Simulando uma base de conhecimento (normalmente seria recuperada de um banco de dados vetorial)
knowledge_base = """
DOCUMENTO 1 - Azure AI Foundry (2024):
O Azure AI Foundry é uma plataforma unificada para desenvolvimento de aplicações de IA generativa. 
Lançado em 2024, oferece ferramentas integradas para treinar, avaliar e implantar modelos de IA.
Inclui recursos como prompt flow, avaliação automatizada e monitoramento de modelos.

DOCUMENTO 2 - Azure OpenAI Service:
O Azure OpenAI Service fornece acesso aos modelos GPT-4, GPT-3.5-turbo, DALL-E e Codex através de APIs REST.
Oferece recursos empresariais como redes virtuais, chaves gerenciadas pelo cliente e compliance.
Disponível em múltiplas regiões com diferentes modelos e capacidades.

DOCUMENTO 3 - Prompt Engineering Best Practices:
Técnicas essenciais incluem few-shot learning, chain-of-thought e prompt chaining.
Importante ser específico, usar exemplos claros e estruturar bem as instruções.
A ordem das informações no prompt pode afetar significativamente os resultados.
"""

# Pergunta do usuário
pergunta = "Quais são as principais características do Azure AI Foundry lançado em 2024?"

# Prompt RAG: Combinando contexto recuperado com a pergunta
rag_prompt = f"""Com base nas informações fornecidas abaixo, responda à pergunta do usuário de forma precisa e cite as fontes relevantes.

CONTEXTO RECUPERADO:
{knowledge_base}

PERGUNTA: {pergunta}

INSTRUÇÕES:
- Use apenas as informações fornecidas no contexto
- Cite o documento específico quando relevante
- Se a informação não estiver disponível no contexto, indique claramente
- Seja preciso e objetivo na resposta"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": rag_prompt}
    ],
    temperature=0.2
)

print("Resposta RAG:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Exemplo 2: Pergunta que não está no contexto
pergunta2 = "Qual é o preço do Azure AI Foundry?"

rag_prompt2 = f"""Com base nas informações fornecidas abaixo, responda à pergunta do usuário.

CONTEXTO RECUPERADO:
{knowledge_base}

PERGUNTA: {pergunta2}

INSTRUÇÕES:
- Use apenas as informações fornecidas no contexto
- Se a informação não estiver disponível, responda: "Informação não encontrada no contexto fornecido"
"""

response2 = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": rag_prompt2}
    ],
    temperature=0.1
)

print("Teste com informação não disponível:")
print(response2.choices[0].message.content)

### Active-Prompt

Active-Prompt é uma técnica que seleciona automaticamente os exemplos mais úteis e informativos para usar em few-shot prompting, baseando-se na incerteza do modelo sobre certas questões.

**Características:**
- Seleção automática de exemplos mais úteis
- Baseada na incerteza do modelo
- Melhora a eficiência do few-shot learning
- Reduz a necessidade de curadoria manual de exemplos

**Como Funciona:**
1. **Análise de Incerteza**: Identifica onde o modelo tem mais dúvidas
2. **Seleção de Exemplos**: Escolhe exemplos que abordam essas incertezas
3. **Prompting Adaptativo**: Usa os exemplos mais relevantes para cada consulta
4. **Refinamento Contínuo**: Melhora com base no feedback

**Exemplo Conceitual:**
```
Tarefa: Classificação de sentimentos

Modelo identifica incerteza em:
- Sarcasmo
- Linguagem ambígua
- Expressões culturais

Active-Prompt seleciona exemplos que abordam especificamente:
- "Que maravilha, choveu no meu casamento!" (Sarcasmo → Negativo)
- "Não sei se gostei..." (Ambíguo → Neutro)
- "Tá de boa!" (Cultural → Positivo)
```

**Quando usar:**
- Quando você tem muitos exemplos disponíveis
- Para otimizar automaticamente few-shot prompts
- Em domínios onde a incerteza varia por tópico
- Para sistemas adaptativos que melhoram com o uso

In [ ]:
# Exemplo prático de Active-Prompt (Simulado)
print("=== TESTE: Active-Prompt ===")

# Simulando o processo de seleção de exemplos baseado na incerteza
def simulate_active_prompt():
    # Pool de exemplos disponíveis
    example_pool = [
        ("Que maravilha, choveu no meu casamento!", "Negativo", "sarcasmo"),
        ("Este produto é ok, nada demais.", "Neutro", "ambiguidade"),
        ("Adorei a experiência!", "Positivo", "direto"),
        ("Tá de boa esse app!", "Positivo", "linguagem_informal"),
        ("Não sei se gostei muito...", "Neutro", "incerteza"),
        ("Simplesmente fantástico!", "Positivo", "entusiasmo"),
        ("Podia ser melhor, né?", "Negativo", "crítica_indireta")
    ]
    
    # Pergunta que gera incerteza (linguagem informal + sarcasmo potencial)
    query = "Tá massa esse bagulho, viu!"
    
    # Active-Prompt seleciona exemplos mais relevantes para casos similares
    selected_examples = [
        ("Que maravilha, choveu no meu casamento!", "Negativo"),  # sarcasmo
        ("Tá de boa esse app!", "Positivo"),  # linguagem informal
        ("Podia ser melhor, né?", "Negativo")  # tom ambíguo
    ]
    
    return query, selected_examples

# Executando simulação
query, selected_examples = simulate_active_prompt()

print("Exemplo de Active-Prompt em ação:")
print(f"Frase para classificar: '{query}'")
print("\nExemplos selecionados automaticamente baseados na incerteza:")
for i, (frase, sentimento) in enumerate(selected_examples, 1):
    print(f"{i}. '{frase}' → {sentimento}")

# Construindo prompt com exemplos selecionados dinamicamente
active_prompt = "Classifique o sentimento (Positivo, Negativo, Neutro) considerando contexto cultural e possível sarcasmo:\n\n"

for frase, sentimento in selected_examples:
    active_prompt += f"Frase: '{frase}' → Sentimento: {sentimento}\n"

active_prompt += f"\nFrase: '{query}' → Sentimento: ?"

print(f"\n{'-'*50}")
print("Prompt construído dinamicamente:")
print(active_prompt)

# Executando classificação
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": active_prompt}
    ],
    temperature=0.1
)

print(f"\n{'-'*50}")
print("Resultado do Active-Prompt:")
print(response.choices[0].message.content)
print("\n" + "="*50 + "\n")

# Comparação: mesmo prompt sem Active-Prompt (exemplos genéricos)
generic_prompt = """Classifique o sentimento (Positivo, Negativo, Neutro):

Frase: 'Eu amo este produto!' → Sentimento: Positivo
Frase: 'Este serviço é terrível.' → Sentimento: Negativo
Frase: 'O produto está ok.' → Sentimento: Neutro

Frase: 'Tá massa esse bagulho, viu!' → Sentimento: ?"""

response_generic = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": generic_prompt}
    ],
    temperature=0.1
)

print("Comparação com Few-Shot genérico:")
print(response_generic.choices[0].message.content)
print("\nNote: Active-Prompt selecionou exemplos mais relevantes para lidar com linguagem informal e possível sarcasmo.")

### Melhores Práticas

**Seja Específico.** Deixe o mínimo possível para interpretação. Restrinja o espaço operacional.

**Seja Descritivo.** Use analogias para tornar as instruções mais claras.

**Reforce as Instruções.** Às vezes pode ser necessário se repetir para o modelo. Forneça instruções antes e depois do seu conteúdo principal, use uma instrução e uma deixa, etc.

**A Ordem Importa.** A ordem em que você apresenta informações ao modelo pode impactar o resultado. Se você coloca instruções antes do seu conteúdo ("resuma o seguinte...") ou depois ("resuma o texto acima...") pode fazer diferença no resultado. Até mesmo a ordem dos exemplos few-shot pode importar. Isso é conhecido como viés de recência.

**Ofereça uma Alternativa ao Modelo.** Às vezes pode ser útil dar ao modelo um caminho alternativo se ele não conseguir completar a tarefa atribuída. Por exemplo, ao fazer uma pergunta sobre um texto, você pode incluir algo como "responda com 'não encontrado' se a resposta não estiver presente." Isso pode ajudar o modelo a evitar gerar respostas falsas.

In [ ]:
# 🧪 ÁREA DE EXPERIMENTAÇÃO
# Use este espaço para testar as técnicas de Prompt Engineering

# Exemplo: Teste sua própria técnica aqui
# response = client.chat.completions.create(
#     model=deployment_name,
#     messages=[
#         {"role": "user", "content": "Seu prompt aqui..."}
#     ],
#     temperature=0.5
# )
# 
# print(response.choices[0].message.content)

# Seu código aqui...